In [1]:
%%capture
!pip install datasets rouge-score nltk transformers sentencepiece

In [75]:
#GPU check
!nvidia-smi

Wed Mar 30 02:00:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    41W / 250W |   4417MiB / 16280MiB |     81%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [74]:
torch.cuda.empty_cache()

In [3]:
# Make sure your version of Transformers is at 4.11.0
# to run the following code correctly:
import datasets
import transformers
import torch

In [4]:
torch_device ='cuda'
#torch_device = 'cpu' (run this line if no gpu attached)

In [5]:
#load original model
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration

In [6]:
tokenizer = ProphetNetTokenizer.from_pretrained("microsoft/prophetnet-large-uncased")
model = ProphetNetForConditionalGeneration.from_pretrained("microsoft/prophetnet-large-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/141 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

In [ ]:
model.to(torch_device)

In [7]:
#prefix = "summarize: "
prefix = ""

##XSUM

In [8]:
#load test dataset
test_dataset = datasets.load_dataset('xsum', split='test')

Using custom data configuration default


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


In [9]:
small_test = test_dataset.select(list(range(2000)))
small_test

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2000
})

In [10]:
# generate summary
batch_size = 8
max_input_length=512
max_target_length=64

def generate_summary(batch):
    sentences = [prefix + text for text in batch['document']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [14]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["document"])

  0%|          | 0/250 [00:00<?, ?ba/s]

In [16]:
results

Dataset({
    features: ['summary', 'id', 'pred'],
    num_rows: 2000
})

In [17]:
metric = datasets.load_metric("rouge")
rouge_score = metric.compute(predictions=results["pred"], references=results["summary"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [18]:
rouge_mean

{'rouge1': 13.399329786557857,
 'rouge2': 1.9954408619922628,
 'rougeL': 10.362052273091713,
 'rougeLsum': 10.359114115980443}

In [33]:
!ls content/

prophetnet-large-uncased-finetuned-xsum


In [28]:
#load fine-tuned model
!cp 'drive/MyDrive/weights/prophetnet-large-uncased-finetuned-xsum.zip' '/content'

In [31]:
!unzip prophetnet-large-uncased-finetuned-xsum.zip

Archive:  prophetnet-large-uncased-finetuned-xsum.zip
   creating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/
  inflating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/special_tokens_map.json  
  inflating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/config.json  
  inflating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/optimizer.pt  
  inflating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/rng_state.pth  
  inflating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/trainer_state.json  
  inflating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/tokenizer_config.json  
  inflating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/pytorch_model.bin  
  inflating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/scaler.pt  
  inflating: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/training_args.bin

In [34]:
finetuned_model = ProphetNetForConditionalGeneration.from_pretrained('content/prophetnet-large-uncased-finetuned-xsum/checkpoint-51012/').to(torch_device)

In [35]:
# generate summary
batch_size = 8
def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['document']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [36]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["document"])

  0%|          | 0/250 [00:00<?, ?ba/s]

In [37]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["summary"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [38]:
rouge_mean

{'rouge1': 13.399329786557857,
 'rouge2': 1.9954408619922628,
 'rougeL': 10.362052273091713,
 'rougeLsum': 10.359114115980443}

In [39]:
finetuned_rouge_mean

{'rouge1': 41.55343853328356,
 'rouge2': 18.176918428697036,
 'rougeL': 33.30889167452633,
 'rougeLsum': 33.30956720015447}

In [40]:
for i in range(20, 30):
    print("Original Text: %s" % small_test[i]['document'])
    print("\nActual Summary: %s" % results[i]['summary'])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: Pakistan's telecoms regulator said the ban was no longer necessary because Google, which owns YouTube, had now launched a Pakistan-specific version.
YouTube has denied claims that the authorities can filter content.
Many young Pakistanis have welcomed the lifting of the ban but some activists want details of the deal with Google.
They say there should be greater transparency of the terms agreed between Google and the government.
A Pakistan Telecommunication Authority (PTA) official confirmed to the BBC that all internet service providers had been directed to open access to YouTube.
The Pakistan Telecommunication Company Ltd posted on its Facebook page on Monday: "Welcome Back YouTube".
Pakistan's ministry of information technology said: "Google has provided an online web process through which requests for blocking access of offending material can be made by the PTA to Google directly.
"Google/YouTube will accordingly restrict access to the said offending material for use

##CNN

In [41]:
#load test dataset
test_cnn = datasets.load_dataset('cnn_dailymail', '3.0.0', split='test')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


In [42]:
small_test = test_cnn.select(list(range(2000)))
small_test

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 2000
})

In [43]:
# generate summary
batch_size = 8
max_input_length=512
max_target_length=128

def generate_summary(batch):
    sentences = [prefix + text for text in batch['article']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [44]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

  0%|          | 0/250 [00:00<?, ?ba/s]

In [45]:
results

Dataset({
    features: ['highlights', 'id', 'pred'],
    num_rows: 2000
})

In [46]:
rouge_score = metric.compute(predictions=results["pred"], references=results["highlights"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [47]:
rouge_mean

{'rouge1': 20.039277735516407,
 'rouge2': 6.012750800312023,
 'rougeL': 14.160081716364953,
 'rougeLsum': 17.294968771231417}

In [48]:
#load fine-tuned model
!cp 'drive/MyDrive/weights/prophetnet-finetuned-cnn-half.zip' '/content'

In [49]:
!unzip prophetnet-finetuned-cnn-half.zip

Archive:  prophetnet-finetuned-cnn-half.zip
   creating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/
  inflating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/special_tokens_map.json  
  inflating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/config.json  
  inflating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/optimizer.pt  
  inflating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/rng_state.pth  
  inflating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/trainer_state.json  
  inflating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/tokenizer_config.json  
  inflating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/pytorch_model.bin  
  inflating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/scaler.pt  
  inflating: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/training_args.bin  
  inflating: cont

In [50]:
finetuned_model = ProphetNetForConditionalGeneration.from_pretrained('content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/').to(torch_device)

In [54]:
# generate summary
batch_size = 8
def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['article']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [55]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

  0%|          | 0/250 [00:00<?, ?ba/s]

In [56]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["highlights"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [57]:
rouge_mean

{'rouge1': 20.039277735516407,
 'rouge2': 6.012750800312023,
 'rougeL': 14.160081716364953,
 'rougeLsum': 17.294968771231417}

In [58]:
finetuned_rouge_mean

{'rouge1': 38.45306192931015,
 'rouge2': 17.016256633658976,
 'rougeL': 27.090304215136474,
 'rougeLsum': 32.41676002728493}

In [59]:
for i in range(0, 15):
    print("Original Text: %s" % small_test[i]["article"])
    print("\nActual Summary: %s" % results[i]["highlights"])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: (CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV's "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he'd been a busy actor for decades in theater and in Hollywood, Best didn't become famous until 1979, when "The Dukes of Hazzard's" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best's Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphras

##Wikihow

In [60]:
#load test dataset
test_wiki = datasets.load_dataset("wikihow", "all", "/content/drive/MyDrive/dataset", split='test')

Using custom data configuration all-6f5101161f12f62f


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/157252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5599 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5577 [00:00<?, ? examples/s]

Dataset wikihow downloaded and prepared to /root/.cache/huggingface/datasets/wikihow/all-6f5101161f12f62f/1.2.0/5343fc81d685acaa086c9cc19eb8706206cd1f8b315792b04c1d7b92091c305e. Subsequent calls will reuse this data.


In [64]:
small_test = test_wiki.select(list(range(2000)))
small_test

Dataset({
    features: ['text', 'headline', 'title'],
    num_rows: 2000
})

In [65]:
# generate summary
batch_size = 8
max_input_length=512
max_target_length=128

def generate_summary(batch):
    sentences = [prefix + text for text in batch['text']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [66]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

  0%|          | 0/250 [00:00<?, ?ba/s]

In [67]:
results

Dataset({
    features: ['headline', 'title', 'pred'],
    num_rows: 2000
})

In [68]:
rouge_score = metric.compute(predictions=results["pred"], references=results["headline"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [69]:
rouge_mean

{'rouge1': 16.796872539813627,
 'rouge2': 3.549800040884704,
 'rougeL': 11.643451038158842,
 'rougeLsum': 15.178905724987036}

In [72]:
!ls 

content				   prophetnet-large-uncased-finetuned-wiki.zip
drive				   prophetnet-large-uncased-finetuned-xsum.zip
prophetnet-finetuned-cnn-half.zip  sample_data


In [71]:
#load fine-tuned model
!cp '/content/drive/My Drive/weights/prophetnet-large-uncased-finetuned-wiki.zip' '/content'

In [73]:
!unzip prophetnet-large-uncased-finetuned-wiki.zip

Archive:  prophetnet-large-uncased-finetuned-wiki.zip
   creating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/
  inflating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/special_tokens_map.json  
  inflating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/config.json  
  inflating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/optimizer.pt  
  inflating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/rng_state.pth  
  inflating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/trainer_state.json  
  inflating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/tokenizer_config.json  
  inflating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/pytorch_model.bin  
  inflating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/scaler.pt  
  inflating: content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/training_args.bin

In [76]:
finetuned_model = ProphetNetForConditionalGeneration.from_pretrained('content/prophetnet-large-uncased-finetuned-wiki/checkpoint-39313/').to(torch_device)

In [77]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['text']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [78]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

  0%|          | 0/250 [00:00<?, ?ba/s]

In [79]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["headline"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [80]:
rouge_mean

{'rouge1': 16.796872539813627,
 'rouge2': 3.549800040884704,
 'rougeL': 11.643451038158842,
 'rougeLsum': 15.178905724987036}

In [81]:
finetuned_rouge_mean

{'rouge1': 37.55479236729412,
 'rouge2': 15.357963332212787,
 'rougeL': 29.55316313193752,
 'rougeLsum': 34.52134093822632}

In [82]:
for i in range(20, 30):
    print("Original Text: %s" % small_test[i]['text'])
    print("\nActual Summary: %s" % results[i]['headline'])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: In general, dwarf hamsters have thickset bodies, large cheek pouches, and short tails. Make sure it does not have any nasal or eye discharge, nor any other signs of illness. Russian dwarf hamsters are particularly prone to diabetes. A hamster suffering from this condition will drink a lot of water and urinate more frequently than other dwarf hamsters.Be sure to consult your veterinarian if you suspect that your hamster has diabetes.
 A recently weaned or an extremely stressed out dwarf hamster may come down with a disease called “wet tail.” Your hamster experiences diarrhea — the excessive moisture from this causes its tail to become literally wet. Consult a veterinarian for a proper diagnosis and treatment.


Tyzzer’s disease causes diarrhea in young or stressed hamsters. This is a disease that needs veterinarian treatment. Certain antibiotics can cause and exacerbate this condition, so don’t treat your hamster on your own., Like dogs and cats, dwarf hamsters can suffer